In [1]:
%cd C:\Users\User\Documents\Spotify Analysis Repos\top200globalchart\spotify-top200globalchart-analysis

C:\Users\User\Documents\Spotify Analysis Repos\top200globalchart\spotify-top200globalchart-analysis


In [2]:
import pandas as pd
import json
import string

In [3]:
all_files_za = pd.read_csv("Classification/all_files_ZAv1.csv")

In [4]:
trackappearancecountdf = all_files_za[all_files_za["artistAppearanceCount"] == 0]

In [5]:
trackappearancecountdf.to_csv("Classification/firstappearanceartistsza.csv", index=False)

In [6]:
list(trackappearancecountdf.columns)

['trackAppearanceCount',
 'artistAppearanceCount',
 'rank',
 'uri',
 'artist_names',
 'track_name',
 'source',
 'peak_rank',
 'previous_rank',
 'weeks_on_chart',
 'streams',
 'End Date',
 'Week',
 'Year',
 'End Date Dt',
 'Seconds since Epoch',
 'End Date Aggregated',
 'ArtistCount',
 'main_artist',
 'isTopTen',
 'Artist and Track',
 'rank difference',
 'Position over Time']

In [7]:
relavantfirsttimers = trackappearancecountdf[["main_artist", "track_name", 'trackAppearanceCount', 'artistAppearanceCount','End Date', "Year",'streams']]

In [8]:
relavantfirsttimerssorted = relavantfirsttimers.sort_values(by=['End Date'],ascending=False)

In [9]:
racing_bar_df = relavantfirsttimerssorted[['End Date', "main_artist", 'Year', 'streams']]
racing_bar_df.columns = ['date', "name", 'category', 'value']

In [10]:
racing_bar_df

,date,name,category,value
12211,2024-03-14,Dasha,2024,74002
21702,2024-03-14,Goldmax,2024,71007
43029,2024-03-14,New Edition,2024,73265
396,2024-03-14,4batz,2024,124187
60594,2024-02-29,Xolly Mncwango,2024,84799
...,...,...,...,...
20289,2018-03-15,Fort Minor,2018,1595
9167,2018-03-15,Cashmere Cat,2018,1198
48856,2018-03-15,Rob $tone,2018,1445
49088,2018-03-15,Rudimental,2018,4479


In [11]:
def getanartistsjson(all_files_za, artist_name):

    artist_df = all_files_za[all_files_za["main_artist"] == artist_name]
    
    num_of_streams = artist_df['streams'].sum()
    min_date = artist_df['End Date'].min()
    max_date = artist_df['End Date'].max()

    unique_songs = artist_df['track_name'].unique().tolist()
    num_unique_songs = len(unique_songs)
    
    source_end_date = artist_df[['End Date', "source"]].drop_duplicates()
    source_end_date_latest = pd.DataFrame(source_end_date.groupby("source")["End Date"].max())
    source_end_date_latest = source_end_date_latest.reset_index()
    
    rank_max_end_date = artist_df[['End Date', 'track_name', "rank"]].drop_duplicates()

    min_rank = rank_max_end_date["rank"].min()
    max_rank = rank_max_end_date["rank"].max()

    min_rank_song_dt = rank_max_end_date[rank_max_end_date["rank"] == min_rank]
    max_rank_song_dt = rank_max_end_date[rank_max_end_date["rank"] == max_rank]

    min_rank_song_unique_dt = min_rank_song_dt[["End Date", "track_name"]].drop_duplicates()
    max_rank_song_unique_dt = max_rank_song_dt[["End Date", "track_name"]].drop_duplicates()

    source_end_date_latest_ls = source_end_date_latest.values.tolist()

    min_rank_song_unique_dt_ls = min_rank_song_unique_dt.values.tolist()
    max_rank_song_unique_dt_ls = max_rank_song_unique_dt.values.tolist()
    
    artist_inner_json = {}
    
    artist_inner_json["num_of_streams"] = str(num_of_streams)
    artist_inner_json["min_rank"] = str(min_rank)
    artist_inner_json["max_rank"] = str(max_rank)
    artist_inner_json["min_date"] = min_date
    artist_inner_json["max_date"] = max_date
    artist_inner_json["num_unique_songs"] = str(num_unique_songs)

    artist_inner_json["unique_songs"] = unique_songs

    artist_inner_json["source_end_date_latest_ls"] = dict(source_end_date_latest_ls)
    artist_inner_json["min_rank_song_unique_dt_ls"] = dict(min_rank_song_unique_dt_ls)
    artist_inner_json["max_rank_song_unique_dt_ls"] = dict(max_rank_song_unique_dt_ls)
    
    artist_json = {}
    artist_json[artist_name] = artist_inner_json
    
    return artist_json

In [12]:
unique_artists = all_files_za["main_artist"].unique().tolist()
all_artists_jsons = {}
    
for artist_name in unique_artists:  
    artist_json = getanartistsjson(all_files_za, artist_name)
    
    all_artists_jsons[artist_name] = artist_json
    # Serializing json
    json_object = json.dumps(artist_json, indent=4)
    
    new_artist_name = artist_name.translate(str.maketrans('', '', string.punctuation))

#     Writing to sample.json
#     with open("za_artists/" + new_artist_name +".json", "w") as outfile:
#         outfile.write(json_object)

### MAKE A SCRIPT for the voice over

In [13]:
max_date = all_files_za["End Date"].max()

In [14]:
max_date

'2024-03-14'

In [15]:
all_files_za_max_date = all_files_za[all_files_za["End Date"] == max_date]

In [16]:
unique_artists_latest = all_files_za_max_date["main_artist"].unique().tolist()

#### Get list of top 10 SA artists to make a script of

In [17]:
def theartiststofocuson(all_files_za,thedate,topten):
    thedatefiles = all_files_za[(all_files_za['End Date']==thedate) & (all_files_za['rank']<=topten)]
    
    artistcountry = pd.read_csv("Classification/uniquartists_cat.csv")
    
    thedatefilesvf = thedatefiles.merge(artistcountry, left_on=["main_artist"], right_on=["main_artist"], how="inner")
    thedatefilesvf = thedatefilesvf[thedatefilesvf["country"]=="ZA"]
    
    thelisttoscriptize = thedatefilesvf["main_artist"].unique().tolist()
    
    return thelisttoscriptize

thedate = "2024-02-29"
topten = 10
thelisttoscriptize = theartiststofocuson(all_files_za,thedate,topten)

In [18]:
thelisttoscriptize = ["Busta 929"]

#### Get list of top 10 remove from videos already made

In [19]:
alreadymadevidsartists = pd.read_csv("Classification/vidsartistshortsalreadymade.csv", encoding = "ISO-8859-1")

In [20]:
alreadymadevidsartistsls = alreadymadevidsartists["Artist"].unique().tolist()

In [21]:
thelisttoscriptizevf = list(set(thelisttoscriptize) - set(alreadymadevidsartistsls))
thelisttoscriptizevf.append("Harry Cane")

In [22]:
thelisttoscriptizevf

['Busta 929', 'Harry Cane']

In [23]:
# artists_of_interest = ['Kabza De Small', 'Blxckie', 'Busta 929', 'Mellow & Sleazy', 'Senior Oat', 'Wanitwa Mos', 'Mr JazziQ', 'Musa Keys', "Mas Musiq", "Deep London"]

In [24]:
artists_of_interest = thelisttoscriptizevf

In [25]:
# artists_of_interest = ['Blxckie']

In [26]:
# artists_of_interest = ["Mas Musiq", "Deep London"]

In [27]:
# need to make changes in the script here......

In [73]:
def formatnums(num):
    if num > 1000000:
        if not num % 1000000:
            return f'{num // 1000000} million'
        return f'{round(num / 1000000, 1)} million'
    return f'{num // 1000} thousand'

In [75]:
for anartist in artists_of_interest:
    voiceover_str = ""
    jsonartist = all_artists_jsons[anartist]
    
    thename =  anartist + " have " + jsonartist[anartist]["num_unique_songs"] +" songs on the Spotify Top 200 Weekly South Africa chart. \n "
    
    minmonth = datetime.datetime.strptime(jsonartist[anartist]["min_date"].split("-")[1], "%m").strftime("%B")
    minday = jsonartist[anartist]["min_date"].split("-")[2]
    minyear = jsonartist[anartist]["min_date"].split("-")[0]
    mindate = minday + " " + minmonth + " "+ minyear
    
    maxmonth = datetime.datetime.strptime(jsonartist[anartist]["max_date"].split("-")[1], "%m").strftime("%B")
    maxday = jsonartist[anartist]["max_date"].split("-")[2]
    maxyear = jsonartist[anartist]["max_date"].split("-")[0]
    maxdate = maxday + " " + maxmonth + " "+ maxyear
    
    appearance_date = "He first appeared on the chart on " + mindate + "." + " His most recent appearance has been on " + maxdate
    
    songs_streams = ". He has a total of " + formatnums(float(jsonartist[anartist]["num_of_streams"])) + " streams. \n"
    
    
    min_rank_songs = list(set(list(jsonartist[anartist]["min_rank_song_unique_dt_ls"].values())))
    min_rank_songs_not_unique = (list(jsonartist[anartist]["min_rank_song_unique_dt_ls"].values()))
    
    max_rank_songs = list(set(list(jsonartist[anartist]["max_rank_song_unique_dt_ls"].values())))
    max_rank_songs_not_unique = list(jsonartist[anartist]["max_rank_song_unique_dt_ls"].values())
    
    min_rank_songs_dts_not_unique = list(jsonartist[anartist]["min_rank_song_unique_dt_ls"].keys())
    max_rank_songs_dts_not_unique = list(jsonartist[anartist]["max_rank_song_unique_dt_ls"].keys())
    
    positions_on_chart = ". His highest position on the chart is " + jsonartist[anartist]["min_rank"] + " for the song "
    if(len(min_rank_songs) == 1):
        min_rank_songs_dt_day = min_rank_songs_dts_not_unique[0].split("-")[2]
        min_rank_songs_dt_year = min_rank_songs_dts_not_unique[0].split("-")[0]
        min_rank_songs_dt_month = datetime.datetime.strptime(min_rank_songs_dts_not_unique[0].split("-")[1], "%m").strftime("%B")
        min_rank_dt = min_rank_songs_dt_day + " " + min_rank_songs_dt_month + " " + min_rank_songs_dt_year
        positions_on_chart +=  min_rank_songs[0] + " which was at this position on " + min_rank_dt
    else:
        for the_dt in min_rank_songs_dts_not_unique:
            thedt_day = jsonartist[anartist]["min_rank_song_unique_dt_ls"][the_dt].split("-")[2]
            thedt_year = jsonartist[anartist]["min_rank_song_unique_dt_ls"][the_dt].split("-")[0]
            thedt_month = datetime.datetime.strptime(jsonartist[anartist]["min_rank_song_unique_dt_ls"][the_dt].split("-")[1], "%m").strftime("%B")
            thedtdate = thedt_day + " " + thedt_month + " " + thedt_year
            
            positions_on_chart += jsonartist[anartist]["min_rank_song_unique_dt_ls"][the_dt] + " which was at this position on " + thedtdate + " "
    
            
    songs_on_chart = " The songs of his that have appeared on the chart are " + " ... ".join(jsonartist[anartist]["unique_songs"])
    
    voiceover_str = thename + appearance_date + songs_streams + positions_on_chart + "\n" + songs_on_chart
    
    voiceover_str_json = {}
    voiceover_str_json[anartist] = voiceover_str
    # Serializing json
    json_object = json.dumps(voiceover_str_json, indent=4)
    
    new_artist_name = anartist.translate(str.maketrans('', '', string.punctuation))

#     Writing to sample.json
    print("za_artists/" + new_artist_name +"_script.json")
    with open("za_artists/" + new_artist_name +"_script.json", "w") as outfile:
        outfile.write(json_object)

za_artists/Busta 929_script.json
za_artists/Harry Cane_script.json


In [53]:
# for anartist in artists_of_interest:
#     voiceover_str = ""
#     jsonartist = all_artists_jsons[anartist]
    
#     thename = "Let us take a look at " + anartist + "'s journey on the Spotify Top 200 Weekly South Africa chart. \n "
#     songs_streams = "He has had a total of " + jsonartist[anartist]["num_unique_songs"] + " songs with approximately " + jsonartist[anartist]["num_of_streams"] + " million streams. \n"
    
#     record_labels = ""
#     if(len(jsonartist[anartist]["source_end_date_latest_ls"].values()) == 1):
#         record_labels = "As of " + max_date + " , he is with " + list(jsonartist[anartist]["source_end_date_latest_ls"].keys())[0]
#     else:
        
#         record_labels = "As of " + max_date + ", he has released music with " + ' .... '.join(list(jsonartist[anartist]["source_end_date_latest_ls"].keys()))
        
#     appearance_date = "He first appeared on the chart on " + jsonartist[anartist]["min_date"] + "." + " His most recent appearance has been on " + jsonartist[anartist]["max_date"]
    
#     min_rank_songs = list(set(list(jsonartist[anartist]["min_rank_song_unique_dt_ls"].values())))
#     min_rank_songs_not_unique = (list(jsonartist[anartist]["min_rank_song_unique_dt_ls"].values()))
    
#     max_rank_songs = list(set(list(jsonartist[anartist]["max_rank_song_unique_dt_ls"].values())))
#     max_rank_songs_not_unique = list(jsonartist[anartist]["max_rank_song_unique_dt_ls"].values())
    
#     min_rank_songs_dts_not_unique = list(jsonartist[anartist]["min_rank_song_unique_dt_ls"].keys())
#     max_rank_songs_dts_not_unique = list(jsonartist[anartist]["max_rank_song_unique_dt_ls"].keys())
    
#     positions_on_chart = "His highest position on the chart is " + jsonartist[anartist]["min_rank"] + " for the song "
#     if(len(min_rank_songs) == 1):
#         positions_on_chart +=  min_rank_songs[0] + " which was at this position on " + min_rank_songs_dts_not_unique[0]
#     else:
#         for the_dt in min_rank_songs_dts_not_unique:
#             positions_on_chart += jsonartist[anartist]["min_rank_song_unique_dt_ls"][the_dt] + " which was at this position on " + the_dt + " "
    
#     positions_on_chart_lowest = "His lowest position on the chart is " + jsonartist[anartist]["max_rank"] + " for the song "
#     if(len(max_rank_songs) == 1):
#         positions_on_chart_lowest +=  max_rank_songs[0] + " which was at this position on " + max_rank_songs_dts_not_unique[0]
#     else:
#         for the_dt in max_rank_songs_dts_not_unique:
#             positions_on_chart_lowest += jsonartist[anartist]["max_rank_song_unique_dt_ls"][the_dt] + " which was at this position on " + the_dt + " "
            
#     songs_on_chart = "The songs of his that have appeared on the chart are " + " ... ".join(jsonartist[anartist]["unique_songs"])
    
#     voiceover_str = thename + songs_streams + record_labels + "\n"+ appearance_date + "\n" + positions_on_chart + "\n" + positions_on_chart_lowest + "\n" + songs_on_chart
    
#     voiceover_str_json = {}
#     voiceover_str_json[anartist] = voiceover_str
#     # Serializing json
#     json_object = json.dumps(voiceover_str_json, indent=4)
    
#     new_artist_name = anartist.translate(str.maketrans('', '', string.punctuation))

# #     Writing to sample.json
#     print("za_artists/" + new_artist_name +"script.json")
#     with open("za_artists/" + new_artist_name +"script.json", "w") as outfile:
#         outfile.write(json_object)